# Oxylabs Search


> [Oxylabs Search](https://github.com/oxylabs/oxylabs-sdk-python) TODO: Desciption 


## Setup
Following the [instruction](https://developers.oxylabs.io/scraper-apis/web-scraper-api/google#getting-started) TODO: Register instructions

The integration lives in the `langchain-community` package.

In [1]:
# %pip install -U langchain-community

In [2]:
import getpass
import os
from pprint import pprint

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

In [3]:
from langchain_community.utilities import OxylabsSearchAPIWrapper


In [4]:
search = OxylabsSearchAPIWrapper()


In [5]:
pprint(search.run("python"), indent=4)

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?num=5&sca_esv=415243b206e29025&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwjPw5Xx6LaJAxUpI7kGHR1OCs8QmxMoAHoECDoQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4: \n'
 '    TITLE: First appeared\n'
 '    CONTENT: 20 February 1991; 33 years ago\n'
 '    FACTOIDS-ITEM-5: \n'

## Number of results
You can use parameters "start_page",  "pages" and "limit" to set the number of results

In [6]:
search = OxylabsSearchAPIWrapper(
    params={
        "start_page": 1,
        "pages": 5,
        "limit": 30
    }
)


In [7]:
pprint(search.run("python"), indent=4)

('  KNOWLEDGE GRAPH: \n'
 '  TITLE: Python\n'
 '  IMAGES ITEMS: \n'
 '    IMAGES-ITEM-1: Redacted base64 image string...\n'
 '    IMAGES-ITEM-2: Redacted base64 image string...\n'
 '    IMAGES-ITEM-3: Redacted base64 image string...\n'
 '  FACTOIDS ITEMS: \n'
 '    FACTOIDS-ITEM-1: \n'
 '    LINKS ITEMS: \n'
 '      LINKS-ITEM-1: \n'
 '      HREF: '
 '/search?num=30&sca_esv=415243b206e29025&gl=us&hl=en&q=Guido+van+Rossum&stick=H4sIAAAAAAAAAONgVuLUz9U3MMwwME1exCrgXpqZkq9QlpinEJRfXFyaCwDpnKYPIAAAAA&sa=X&ved=2ahUKEwiLtqf36LaJAxUfqVYBHcfzC7wQmxMoAHoECEYQAg\n'
 '      TITLE: Guido van Rossum\n'
 '    TITLE: Designed by\n'
 '    CONTENT: Guido van Rossum\n'
 '    FACTOIDS-ITEM-2: \n'
 '    TITLE: Developer\n'
 '    CONTENT: Python Software Foundation\n'
 '    FACTOIDS-ITEM-3: \n'
 '    TITLE: Filename extensions\n'
 '    CONTENT: .py,.pyw,.pyz,.pyi,.pyc,.pyd\n'
 '    FACTOIDS-ITEM-4: \n'
 '    TITLE: First appeared\n'
 '    CONTENT: 20 February 1991; 33 years ago\n'
 '    FACTOIDS-ITEM-5: \n

## Metadata Results

In [8]:
search = OxylabsSearchAPIWrapper()


In [9]:
search.results("apples")


[{'paid': [],
  'images': {'items': [{'alt': 'How to Pick the Best Apples | SELF',
     'pos': 1,
     'url': 'https://www.self.com/story/how-to-pick-the-best-apples',
     'data': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='},
    {'alt': 'Fruit of the month: Apples - Harvard Health',
     'pos': 2,
     'url': 'https://www.health.harvard.edu/heart-health/fruit-of-the-month-apples',
     'data': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='},
    {'alt': 'Types of Apples and Apple Facts | The Table by Harry & David',
     'pos': 3,
     'url': 'https://www.harryanddavid.com/articles/food-drink/types-of-apples',
     'data': 'data:image/gif;base64,R0lGODlhAQABAIAAAP///////yH5BAEKAAEALAAAAAABAAEAAAICTAEAOw=='},
    {'alt': 'Varieties Archive - New York Apple Association',
     'pos': 4,
     'url': 'https://www.applesfromny.com/varieties/',
     'data': 'data:image/gif;base64,R0lGODlhAQABAID/AMDAwAAAACH5BAEAA

In [10]:
## Tool Usage

In [11]:
import os

from langchain_community.tools.oxylabs_search import OxylabsSearchResults, OxylabsSearchRun
from langchain_community.utilities import OxylabsSearchAPIWrapper

api_wrapper = OxylabsSearchAPIWrapper()


In [12]:
tool_results = OxylabsSearchResults(wrapper=api_wrapper)
tool_results

OxylabsSearchResults(wrapper=OxylabsSearchAPIWrapper(include_binary_image_data=False, result_categories=[], search_engine=<oxylabs.sources.serp.google.google.Google object at 0x73d0701425f0>, params={'source': 'google_search', 'user_agent_type': 'desktop', 'render': 'html', 'domain': 'com', 'start_page': 1, 'pages': 1, 'limit': 5, 'parse': True, 'locale': '', 'geo_location': '', 'parsing_instructions': {}, 'context': [], 'request_timeout': 165}, oxylabs_username='telesoftas-test', oxylabs_password='e2dKEh7HngSm_AGzQ2Dyt'), kwargs={})

In [13]:
tool_run = OxylabsSearchRun(wrapper=api_wrapper)
tool_run

OxylabsSearchRun(wrapper=OxylabsSearchAPIWrapper(include_binary_image_data=False, result_categories=[], search_engine=<oxylabs.sources.serp.google.google.Google object at 0x73d0701425f0>, params={'source': 'google_search', 'user_agent_type': 'desktop', 'render': 'html', 'domain': 'com', 'start_page': 1, 'pages': 1, 'limit': 5, 'parse': True, 'locale': '', 'geo_location': '', 'parsing_instructions': {}, 'context': [], 'request_timeout': 165}, oxylabs_username='telesoftas-test', oxylabs_password='e2dKEh7HngSm_AGzQ2Dyt'), kwargs={})

In [14]:
import json

# .invoke wraps utility.results
response_results = tool_results.invoke({
    "query": "What is the weather in Shanghai?",
    "geo_location": "",
    "pages": 1,
    "limit": 10
})
response_results = json.loads(response_results)
for item in response_results:
    print(item)

{'paid': [], 'organic': [{'pos': 1, 'url': 'https://www.tripstoshanghai.com/shanghai/weather.htm#:~:text=January%20and%20February%20are%20the,by%20no%20means%20the%20norm.', 'desc': '', 'title': 'Shanghai Weather & Climate | Best Time to Visit Shanghai', 'url_shown': 'https://www.tripstoshanghai.com › shanghai › weather', 'pos_overall': 2, 'favicon_text': 'Shanghai Tours'}, {'pos': 2, 'url': 'https://www.guilinchina.net/travel-guide/shanghai/weather.htm#:~:text=The%20climate%20in%20Shanghai%20is,C%20(60%C2%B0F).', 'desc': '', 'title': 'Shanghai Weather - Guilin China', 'url_shown': 'https://www.guilinchina.net › travel-guide › weather', 'pos_overall': 3, 'favicon_text': 'Guilin China'}, {'pos': 3, 'url': 'https://en.wikipedia.org/wiki/Shanghai#:~:text=Shanghai%20has%20a%20humid%20subtropical,city%20experiences%20four%20distinct%20seasons', 'desc': '', 'title': 'Shanghai - Wikipedia', 'url_shown': 'https://en.wikipedia.org › wiki › Shanghai', 'pos_overall': 4, 'favicon_text': 'Wikipedia

In [15]:
# .invoke wraps utility.results
response_run = tool_run.invoke({
    "query": "What is the weather in Shanghai?",
    "geo_location": "",
    "pages": 1,
    "limit": 10
})

pprint(response_run)

('  ORGANIC RESULTS ITEMS: \n'
 '    ORGANIC-ITEM-1: \n'
 '    POS: 1\n'
 '    URL: '
 'https://www.accuweather.com/en/cn/shanghai/106577/weather-forecast/106577\n'
 '    DESC: Shanghai, Shanghai, China Weather Forecast, with current '
 'conditions, wind, air quality, and what to expect for the next 3 days.\n'
 '    TITLE: Shanghai, Shanghai, China Weather Forecast\n'
 '    SITELINKS: \n'
 '      SITELINKS: \n'
 '      INLINE ITEMS: \n'
 '        INLINE-ITEM-1: \n'
 '        URL: '
 'https://www.accuweather.com/en/cn/shanghai/106577/daily-weather-forecast/106577\n'
 '        TITLE: Daily\n'
 '        INLINE-ITEM-2: \n'
 '        URL: '
 'https://www.accuweather.com/en/cn/shanghai/106577/current-weather/106577\n'
 '        TITLE: Current Weather\n'
 '        INLINE-ITEM-3: \n'
 '        URL: '
 'https://www.accuweather.com/en/cn/shanghai/106577/hourly-weather-forecast/106577\n'
 '        TITLE: Hourly\n'
 '        INLINE-ITEM-4: \n'
 '        URL: '
 'https://www.accuweather.com/en/cn/s

## Chaining


In [16]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai langchainhub langchain-community


[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
# Import necessary modules
import getpass
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI  # Use OpenAIChat instead of AzureChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

# Define tools and agent setup
tool = OxylabsSearchRun(wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

/home/karolisk/workspace/langchain/.venv/lib/python3.10/site-packages/langsmith/client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'latest Burning Man floods', 'geo_location': 'Nevada,United States', 'limit': 3}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://www.outsideonline.com/outdoor-adventure/exploration-survival/burning-man-2023-flooding-rain/
    DESC: Sep 6, 2023 — When storms left Burning Man 2023 a muddy mess, attendees plotted their way out.
    TITLE: How One Man Escaped From Burning Man 2023's Floods
    URL_SHOWN: https://www.outsideonline.com› exploration-survival › b...
    FAVICON_TEXT: Outside Magazine
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://abcnews.go.com/US/burning-man-flooding-happened-stranded-festivalgoers/story?id=102908331
    DESC: Sep 4, 2023 — Tens of thousands of Burning Man attendees are now able to leave the festival after a downpour and massive flooding left them stranded over the weekend.
    TITLE: Burning Man flooding: What happened to stranded ...
    URL_SHOWN: https://abcnews.go.com› story
    

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


During the latest Burning Man event, storms caused flooding and left the festival grounds in a muddy mess. Attendees had to navigate their way out of the flooded area. Tens of thousands of festival attendees were stranded due to the downpour and massive flooding, but they were eventually able to leave the festival. If you'd like more detailed information, I can look up additional sources.

> Finished chain.


{'input': 'What happened in the latest Burning Man floods?',
 'output': "During the latest Burning Man event, storms caused flooding and left the festival grounds in a muddy mess. Attendees had to navigate their way out of the flooded area. Tens of thousands of festival attendees were stranded due to the downpour and massive flooding, but they were eventually able to leave the festival. If you'd like more detailed information, I can look up additional sources."}

In [19]:
agent_executor.invoke({"input": "Who won latest 2024 Lithuanian elections and why, please make an essay from search results."})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': '2024 Lithuanian election results', 'pages': 2}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election
    DESC: 13 hours ago — Presidential elections were held in Lithuania on 12 May 2024, alongside a referendum on allowing multiple citizenships. Incumbent President Gitanas Nausėda ...
    TITLE: 2024 Lithuanian presidential election
    SITELINKS: 
      SITELINKS: 
      INLINE ITEMS: 
        INLINE-ITEM-1: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Background
        TITLE: Background
        INLINE-ITEM-2: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Candidates
        TITLE: Candidates
        INLINE-ITEM-3: 
        URL: https://en.wikipedia.org/wiki/2024_Lithuanian_presidential_election#Campaign
        TITLE: Campaign
        INLINE-ITEM-4: 
        URL: https://en.wikipedia.o

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In the latest 2024 Lithuanian parliamentary elections, the opposition Social Democrats emerged victorious, claiming a significant win in the parliamentary election. The election, which took place on October 27th, saw the Social Democrats securing 52 seats in the 141-seat parliament, marking the end of the four-year rule of the conservative Homeland Union government.

The victory of the Social Democrats in Lithuania's parliamentary election came as a surprise to many, as they managed to secure a majority of seats and pave the way for potential coalition talks. The ruling conservatives conceded defeat as the opposition party led by the Social Democrats garnered support from the electorate.

The election results highlighted the frustration among voters with the previous government and its policies, leading to a shift in power towards the opposition. The Social Democrats' win signifies a significant change in the political landscape of Lithuania and sets the stage for new governance and po

{'input': 'Who won latest 2024 Lithuanian elections and why, please make an essay from search results.',
 'output': "In the latest 2024 Lithuanian parliamentary elections, the opposition Social Democrats emerged victorious, claiming a significant win in the parliamentary election. The election, which took place on October 27th, saw the Social Democrats securing 52 seats in the 141-seat parliament, marking the end of the four-year rule of the conservative Homeland Union government.\n\nThe victory of the Social Democrats in Lithuania's parliamentary election came as a surprise to many, as they managed to secure a majority of seats and pave the way for potential coalition talks. The ruling conservatives conceded defeat as the opposition party led by the Social Democrats garnered support from the electorate.\n\nThe election results highlighted the frustration among voters with the previous government and its policies, leading to a shift in power towards the opposition. The Social Democrats

In [20]:
agent_executor.invoke({"input": "What is the most profitable company in lithuania in 2024?"})



> Entering new AgentExecutor chain...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Invoking: `oxylabs_search` with `{'query': 'most profitable company in Lithuania 2024', 'geo_location': 'Lithuania'}`


  ORGANIC RESULTS ITEMS: 
    ORGANIC-ITEM-1: 
    POS: 1
    URL: https://ceoworld.biz/2024/02/19/lithuanias-largest-companies-by-market-capitalization-2024/
    DESC: Feb 19, 2024 — As of October 24, 2024, AB IGNITIS GRUPE is the most valuable company in Lithuania, with a market cap of $1.51 billion. Following it are TELIA ...
    TITLE: Lithuania's Largest Companies by market capitalization, ...
    URL_SHOWN: https://ceoworld.biz› Latest › Banking and Finance
    FAVICON_TEXT: CEOWORLD magazine
    ORGANIC-ITEM-2: 
    POS: 2
    URL: https://lithuania.lt/news/business-and-innovations-in-lithuania/which-startups-will-be-making-waves-in-lithuania-in-2024/
    DESC: Jan 23, 2024 — Which startups will be making waves in Lithuania in 2024? · 1. Cast AI: · 2. Amlyze: · 3. Vinted Go: · 4. PV Case: · 5. Atrandi Biosciences: · 6.
    TITLE: Which startups will be making 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE with a market capitalization of $1.51 billion.

> Finished chain.


{'input': 'What is the most profitable company in lithuania in 2024?',
 'output': 'The most valuable company in Lithuania in 2024 is AB IGNITIS GRUPE with a market capitalization of $1.51 billion.'}